In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
import random
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from sklearn.cluster import KMeans
from tqdm import tqdm
import cv2
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.preprocessing import LabelEncoder
from PIL import Image

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Define dataset paths
ORIGINAL_TRAIN_PATH = "/content/drive/MyDrive/Cavity Dataset/train/"
ORIGINAL_TEST_PATH = "/content/drive/MyDrive/Cavity Dataset/test/"
COPIED_TRAIN_PATH = "dataset_col/train/"
COPIED_TEST_PATH = "dataset_col/test/"
TARGET_COUNT = 400  # Target number of images per class

# Function to check if an image is valid (not corrupted)
def is_valid_image(img_path):
    try:
        img = cv2.imread(img_path)
        return img is not None
    except Exception:
        return False

# Function to copy dataset while removing corrupted images
def copy_dataset(src, dest):
    if os.path.exists(dest):
        shutil.rmtree(dest)  # Remove existing copy
    os.makedirs(dest, exist_ok=True)

    for class_folder in os.listdir(src):
        src_class_path = os.path.join(src, class_folder)
        dest_class_path = os.path.join(dest, class_folder)
        os.makedirs(dest_class_path, exist_ok=True)

        for filename in os.listdir(src_class_path):
            img_path = os.path.join(src_class_path, filename)
            if is_valid_image(img_path):
                shutil.copy(img_path, dest_class_path)

# Create a copy of the dataset, removing corrupted images
copy_dataset(ORIGINAL_TRAIN_PATH, COPIED_TRAIN_PATH)
copy_dataset(ORIGINAL_TEST_PATH, COPIED_TEST_PATH)

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load copied dataset
train_dataset = ImageFolder(root=COPIED_TRAIN_PATH, transform=transform)

# Load pre-trained ResNet50 as feature extractor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])  # Remove final classification layer
model.to(device)
model.eval()

# Function to extract features
def extract_features(dataset, indices):
    dataloader = DataLoader(Subset(dataset, indices), batch_size=32, shuffle=False)
    features = []
    with torch.no_grad():
        for images, _ in tqdm(dataloader, desc="Extracting Features"):
            images = images.to(device)
            outputs = model(images)
            features.extend(outputs.squeeze().cpu().numpy())
    return np.array(features)

# Perform Adaptive Sampling via Clustering for both classes
def adaptive_oversampling(class_name):
    class_path = os.path.join(COPIED_TRAIN_PATH, class_name)
    images = os.listdir(class_path)
    image_paths = [os.path.join(class_path, img) for img in images]

    # Extract features for clustering
    indices = [i for i, (path, label) in enumerate(train_dataset.samples) if label == train_dataset.class_to_idx[class_name]]
    features = extract_features(train_dataset, indices)

    # Perform K-Means clustering
    n_clusters = 5  # You can change this
    kmeans = KMeans(n_clusters=n_clusters, random_state=SEED, n_init=10)
    cluster_labels = kmeans.fit_predict(features)

    # Count samples per cluster and find the smallest
    cluster_counts = np.bincount(cluster_labels)
    min_cluster = np.argmin(cluster_counts)

    # Get paths of images in the minority cluster
    minority_cluster_paths = [image_paths[i] for i in range(len(image_paths)) if cluster_labels[i] == min_cluster]

    # Oversample minority cluster images first
    extra_needed = TARGET_COUNT - len(images)
    oversample_images = minority_cluster_paths * (extra_needed // len(minority_cluster_paths)) + random.choices(minority_cluster_paths, k=extra_needed % len(minority_cluster_paths))

    for i, img_path in enumerate(oversample_images):
        filename = os.path.basename(img_path)
        new_filename = f"oversampled_{i}_{filename}"
        shutil.copy(img_path, os.path.join(class_path, new_filename))

# Apply adaptive oversampling to both classes separately
for class_name in train_dataset.classes:
    adaptive_oversampling(class_name)

# Extract features and labels for SMOTE/ADASYN
def get_features_and_labels(dataset):
    indices = list(range(len(dataset)))
    features = extract_features(dataset, indices)
    labels = [label for _, label in dataset.samples]
    return np.array(features), np.array(labels)

features, labels = get_features_and_labels(train_dataset)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Apply SMOTE or ADASYN
sampler = SMOTE(random_state=SEED)  # Change to ADASYN() if needed
resampled_features, resampled_labels = sampler.fit_resample(features, encoded_labels)

# Decode labels back to class names
resampled_labels = label_encoder.inverse_transform(resampled_labels)

# Function to generate synthetic images (Placeholder for actual reconstruction)
def generate_synthetic_images(features, labels, dataset, target_path):
    class_dirs = {cls: os.path.join(target_path, cls) for cls in dataset.classes}

    for class_dir in class_dirs.values():
        os.makedirs(class_dir, exist_ok=True)

    for i, (feat, label) in enumerate(zip(features, labels)):
        class_name = dataset.classes[label]
        save_path = os.path.join(class_dirs[class_name], f"synthetic_{i}.png")

        # Convert feature vector into an image (Placeholder: replace with proper inversion)
        img = np.random.rand(224, 224, 3) * 255  # Dummy image, replace with actual reconstruction
        img = Image.fromarray(img.astype(np.uint8))
        img.save(save_path)

# Generate synthetic images and save them
generate_synthetic_images(resampled_features, resampled_labels, train_dataset, COPIED_TRAIN_PATH)

# Final class counts
final_counts = {cls: len(os.listdir(os.path.join(COPIED_TRAIN_PATH, cls))) for cls in train_dataset.classes}
print(f"Final balanced class counts: {final_counts}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 94.1MB/s]
Extracting Features: 100%|██████████| 23/23 [00:10<00:00,  2.21it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated i

Final balanced class counts: {'cavity': 788, 'no_cavity': 788}


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from torch.utils.data import DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Pre-trained Models with Correct Weights
resnet18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
inception = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)

# Remove final classification layers
resnet18 = nn.Sequential(*list(resnet18.children())[:-1])  # ResNet outputs (batch, 512, 1, 1)
inception.fc = nn.Identity()  # Remove Inception's classification layer

# Move models to device
resnet18.to(device).eval()
inception.to(device).eval()

# Define Image Transformations (InceptionNet requires 299x299 input size)
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Training and Test Data
train_dataset = datasets.ImageFolder(root="dataset_col/train/", transform=transform)
test_dataset = datasets.ImageFolder(root="dataset_col/test/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to extract features from both models
def extract_features(loader):
    features = []
    labels = []

    with torch.no_grad():
        for images, targets in tqdm(loader, desc="Extracting Features"):
            images = images.to(device)

            # Extract features
            resnet_features = resnet18(images).squeeze(-1).squeeze(-1)  # Shape: (batch, 512)
            inception_features = inception(images)  # Shape: (batch, 2048)

            # Flatten and concatenate features
            resnet_features = resnet_features.cpu().numpy()
            inception_features = inception_features.cpu().numpy()

            combined_features = np.hstack((resnet_features, inception_features))

            # Store features and labels
            features.extend(combined_features)
            labels.extend(targets.cpu().numpy())

    return np.array(features), np.array(labels)

# Extract features for training and testing
X_train, y_train = extract_features(train_loader)
X_test, y_test = extract_features(test_loader)

# Train Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Make Predictions
y_pred = rf_classifier.predict(X_test)

# Generate Classification Report
report = classification_report(y_test, y_pred, target_names=train_dataset.classes)
print("\nClassification Report:\n", report)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 112MB/s]
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 110MB/s] 
Extracting Features:  83%|████████▎ | 5/6 [00:02<00:00,  1.77it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
Extracting Features: 100%|██████████| 6/6 [00:03<00:00,  1.99it/s]



Classification Report:
               precision    recall  f1-score   support

      cavity       0.86      0.92      0.89        97
   no_cavity       0.89      0.81      0.85        79

    accuracy                           0.87       176
   macro avg       0.87      0.86      0.87       176
weighted avg       0.87      0.87      0.87       176



In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from torch.utils.data import DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Pre-trained Models
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
densenet121 = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)

# Modify Models - Remove Final Classification Layers
resnet50 = nn.Sequential(*list(resnet50.children())[:-1])  # Remove FC layer
densenet121.classifier = nn.Identity()  # Remove classifier

# Move models to device
resnet50.to(device).eval()
densenet121.to(device).eval()

# Define Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Standard size for both models
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Training and Test Data
train_dataset = datasets.ImageFolder(root="dataset_col/train/", transform=transform)
test_dataset = datasets.ImageFolder(root="dataset_col/test/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to Extract Features from Both Models
def extract_features(loader):
    features, labels = [], []
    with torch.no_grad():
        for images, targets in tqdm(loader, desc="Extracting Features"):
            images = images.to(device)

            # Extract features
            resnet_feats = resnet50(images).squeeze(-1).squeeze(-1).cpu().numpy()  # Shape: (batch, 2048)
            densenet_feats = densenet121(images).cpu().numpy()  # Shape: (batch, 1024)

            # Flatten and concatenate features
            combined_features = np.hstack((resnet_feats, densenet_feats))

            # Store features and labels
            features.extend(combined_features)
            labels.extend(targets.cpu().numpy())

    return np.array(features), np.array(labels)

# Extract features for training and testing
X_train, y_train = extract_features(train_loader)
X_test, y_test = extract_features(test_loader)

# Train Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Make Predictions
y_pred = rf_classifier.predict(X_test)

# Generate Classification Report
report = classification_report(y_test, y_pred, target_names=train_dataset.classes)
print("\nClassification Report:\n", report)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 153MB/s]
Extracting Features:  83%|████████▎ | 5/6 [00:02<00:00,  1.72it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
Extracting Features: 100%|██████████| 6/6 [00:03<00:00,  1.94it/s]



Classification Report:
               precision    recall  f1-score   support

      cavity       0.81      0.94      0.87        97
   no_cavity       0.90      0.72      0.80        79

    accuracy                           0.84       176
   macro avg       0.86      0.83      0.83       176
weighted avg       0.85      0.84      0.84       176



In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from torch.utils.data import DataLoader
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from tqdm import tqdm

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Pre-trained Models
inception = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
efficientnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# Modify Models - Remove Final Classification Layers
inception.fc = nn.Identity()  # Remove classification layer
efficientnet.classifier = nn.Identity()  # Remove classification layer

# Move models to device
inception.to(device).eval()
efficientnet.to(device).eval()

# Define Image Transformations (Inception requires 299x299 input)
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Needed for InceptionV3
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Training and Test Data
train_dataset = datasets.ImageFolder(root="dataset_col/train/", transform=transform)
test_dataset = datasets.ImageFolder(root="dataset_col/test/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to Extract Features from Both Models
def extract_features(loader):
    features, labels = [], []
    with torch.no_grad():
        for images, targets in tqdm(loader, desc="Extracting Features"):
            images = images.to(device)

            # Extract features
            inception_feats = inception(images).cpu().numpy()  # Shape: (batch, 2048)
            efficientnet_feats = efficientnet(images).cpu().numpy()  # Shape: (batch, 1280)

            # Flatten and concatenate features
            combined_features = np.hstack((inception_feats, efficientnet_feats))

            # Store features and labels
            features.extend(combined_features)
            labels.extend(targets.cpu().numpy())

    return np.array(features), np.array(labels)

# Extract features for training and testing
X_train, y_train = extract_features(train_loader)
X_test, y_test = extract_features(test_loader)

# Train Support Vector Machine (SVM)
svm_classifier = SVC(kernel='rbf', C=1.0)  # Radial Basis Function kernel
svm_classifier.fit(X_train, y_train)

# Make Predictions
y_pred = svm_classifier.predict(X_test)

# Generate Classification Report
report = classification_report(y_test, y_pred, target_names=train_dataset.classes)
print("\nClassification Report:\n", report)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 93.7MB/s]
Extracting Features:  83%|████████▎ | 5/6 [00:02<00:00,  1.70it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
Extracting Features: 100%|██████████| 6/6 [00:03<00:00,  1.92it/s]



Classification Report:
               precision    recall  f1-score   support

      cavity       0.85      0.91      0.88        97
   no_cavity       0.88      0.80      0.83        79

    accuracy                           0.86       176
   macro avg       0.86      0.85      0.86       176
weighted avg       0.86      0.86      0.86       176



In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from torch.utils.data import DataLoader
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Pre-trained Models
mobilenet = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)
densenet = models.densenet169(weights=models.DenseNet169_Weights.IMAGENET1K_V1)

# Modify Models - Remove Final Classification Layers
mobilenet.classifier = nn.Identity()  # Remove classification layer
densenet.classifier = nn.Identity()  # Remove classification layer

# Move models to device
mobilenet.to(device).eval()
densenet.to(device).eval()

# Define Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Standard input size for both models
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Training and Test Data
train_dataset = datasets.ImageFolder(root="dataset_col/train/", transform=transform)
test_dataset = datasets.ImageFolder(root="dataset_col/test/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to Extract Features from Both Models
def extract_features(loader):
    features, labels = [], []
    with torch.no_grad():
        for images, targets in tqdm(loader, desc="Extracting Features"):
            images = images.to(device)

            # Extract features
            mobilenet_feats = mobilenet(images).cpu().numpy()  # Shape: (batch, 1280)
            densenet_feats = densenet(images).cpu().numpy()  # Shape: (batch, 1664)

            # Flatten and concatenate features
            combined_features = np.hstack((mobilenet_feats, densenet_feats))

            # Store features and labels
            features.extend(combined_features)
            labels.extend(targets.cpu().numpy())

    return np.array(features), np.array(labels)

# Extract features for training and testing
X_train, y_train = extract_features(train_loader)
X_test, y_test = extract_features(test_loader)

# Train K-Nearest Neighbors (KNN) Classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5, metric='euclidean')  # Using 5 neighbors with Euclidean distance
knn_classifier.fit(X_train, y_train)

# Make Predictions
y_pred = knn_classifier.predict(X_test)

# Generate Classification Report
report = classification_report(y_test, y_pred, target_names=train_dataset.classes)
print("\nClassification Report:\n", report)


Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 85.4MB/s]
Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth
100%|██████████| 54.7M/54.7M [00:00<00:00, 130MB/s]
Extracting Features:  83%|████████▎ | 5/6 [00:03<00:00,  1.46it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
Extracting Features: 100%|██████████| 6/6 [00:03<00:00,  1.68it/s]


Classification Report:
               precision    recall  f1-score   support

      cavity       0.83      0.81      0.82        97
   no_cavity       0.78      0.80      0.79        79

    accuracy                           0.81       176
   macro avg       0.80      0.81      0.81       176
weighted avg       0.81      0.81      0.81       176



In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from torch.utils.data import DataLoader
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Pre-trained Models
vit = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)
resnet18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Modify Models - Remove Final Classification Layers
vit.heads = nn.Identity()  # Remove classification layer
resnet18 = nn.Sequential(*list(resnet18.children())[:-1])  # Remove last classification layer

# Move models to device
vit.to(device).eval()
resnet18.to(device).eval()

# Define Image Transformations (ViT requires 224x224 input size)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Required for ViT and ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Training and Test Data
train_dataset = datasets.ImageFolder(root="dataset_col/train/", transform=transform)
test_dataset = datasets.ImageFolder(root="dataset_col/test/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to Extract Features from Both Models
def extract_features(loader):
    features, labels = [], []
    with torch.no_grad():
        for images, targets in tqdm(loader, desc="Extracting Features"):
            images = images.to(device)

            # Extract features
            vit_feats = vit(images).cpu().numpy()  # Shape: (batch, 768)
            resnet_feats = resnet18(images).squeeze(-1).squeeze(-1).cpu().numpy()  # Shape: (batch, 512)

            # Flatten and concatenate features
            combined_features = np.hstack((vit_feats, resnet_feats))

            # Store features and labels
            features.extend(combined_features)
            labels.extend(targets.cpu().numpy())

    return np.array(features), np.array(labels)

# Extract features for training and testing
X_train, y_train = extract_features(train_loader)
X_test, y_test = extract_features(test_loader)

# Train XGBoost Classifier
xgb_classifier = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, use_label_encoder=False, eval_metric="mlogloss")
xgb_classifier.fit(X_train, y_train)

# Make Predictions
y_pred = xgb_classifier.predict(X_test)

# Generate Classification Report
report = classification_report(y_test, y_pred, target_names=train_dataset.classes)
print("\nClassification Report:\n", report)


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:03<00:00, 115MB/s] 
Extracting Features:  83%|████████▎ | 5/6 [00:03<00:00,  1.35it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
Extracting Features: 100%|██████████| 6/6 [00:03<00:00,  1.51it/s]
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:43:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Classification Report:
               precision    recall  f1-score   support

      cavity       0.81      0.88      0.84        97
   no_cavity       0.83      0.75      0.79        79

    accuracy                           0.82       176
   macro avg       0.82      0.81      0.81       176
weighted avg       0.82      0.82      0.82       176



In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from torch.utils.data import DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Pre-trained Models
inception = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
resnet18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Modify Models - Remove Final Classification Layers
inception.fc = nn.Identity()  # Remove classification layer
resnet18 = nn.Sequential(*list(resnet18.children())[:-1])  # Remove last classification layer

# Move models to device
inception.to(device).eval()
resnet18.to(device).eval()

# Define Image Transformations (Inception requires 299x299 input size)
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Required for InceptionV3
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Training and Test Data
train_dataset = datasets.ImageFolder(root="dataset_col/train/", transform=transform)
test_dataset = datasets.ImageFolder(root="dataset_col/test/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to Extract Features from Both Models
def extract_features(loader):
    features, labels = [], []
    with torch.no_grad():
        for images, targets in tqdm(loader, desc="Extracting Features"):
            images = images.to(device)

            # Extract features
            inception_feats = inception(images).cpu().numpy()  # Shape: (batch, 2048)
            resnet_feats = resnet18(images).squeeze(-1).squeeze(-1).cpu().numpy()  # Shape: (batch, 512)

            # Flatten and concatenate features
            combined_features = np.hstack((inception_feats, resnet_feats))

            # Store features and labels
            features.extend(combined_features)
            labels.extend(targets.cpu().numpy())

    return np.array(features), np.array(labels)

# Extract features for training and testing
X_train, y_train = extract_features(train_loader)
X_test, y_test = extract_features(test_loader)

# First Classifier: Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
X_train_rf = rf_classifier.predict_proba(X_train)
X_test_rf = rf_classifier.predict_proba(X_test)

# Second Classifier: Support Vector Machine (SVM)
svm_classifier = SVC(probability=True, kernel='rbf')
svm_classifier.fit(X_train_rf, y_train)
X_train_svm = svm_classifier.predict_proba(X_train_rf)
X_test_svm = svm_classifier.predict_proba(X_test_rf)

# Third Classifier: XGBoost
xgb_classifier = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, use_label_encoder=False, eval_metric="mlogloss")
xgb_classifier.fit(X_train_svm, y_train)

# Make Predictions
y_pred = xgb_classifier.predict(X_test_svm)

# Generate Classification Report
report = classification_report(y_test, y_pred, target_names=train_dataset.classes)
print("\nClassification Report:\n", report)


Extracting Features:  83%|████████▎ | 5/6 [00:02<00:00,  1.79it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
Extracting Features: 100%|██████████| 6/6 [00:02<00:00,  2.02it/s]



Classification Report:
               precision    recall  f1-score   support

      cavity       0.91      0.77      0.84        97
   no_cavity       0.77      0.91      0.83        79

    accuracy                           0.84       176
   macro avg       0.84      0.84      0.84       176
weighted avg       0.85      0.84      0.84       176



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:46:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
